# d-separation

`d-separation` stands for `direction-separation` and it is a `rule` for reading off conditional independence relationships in a DAG. 


In [1]:
import networkx as nx

def get_dag(nodes, edges):
    g = nx.DiGraph()
    
    _ = [g.add_node(n) for n in nodes]
    _ = [g.add_edge(p, c) for p, c in edges]
    
    return g

nodes = ['D', 'I', 'G', 'S', 'L']
edges = [('D', 'G'), ('I', 'G'), ('I', 'S'), ('G', 'L')]

g = get_dag(nodes, edges)

In [2]:
g.nodes()

NodeView(('D', 'I', 'G', 'S', 'L'))

In [3]:
g.edges()

OutEdgeView([('D', 'G'), ('I', 'G'), ('I', 'S'), ('G', 'L')])

In [20]:
from networkx.algorithms.dag import descendants
from networkx.algorithms.simple_paths import all_simple_paths

def get_paths(g, source, target):
    return all_simple_paths(g.to_undirected(), source, target)

def get_descendants(g):
    return {n: list(descendants(g, n)) for n in g.nodes()}

def is_path_active(g, path, Z, descendants):
    def get_triplet(i):
        return path[i-1], path[i], path[i+1]
    
    def is_collider(x, z, y):
        if g.has_edge(x, z) and g.has_edge(y, z):
            return True
        return False
    
    def is_collider_active(z):
        if z in Z:
            return True
        if len(set(descendants[z]) & set(Z)) > 0:
            return True
        return False
    
    def is_noncollider_active(z):
        if z in Z:
            return False
        return True
    
    def is_node_active(x, z, y):
        if is_collider(x, z, y):
            return is_collider_active(z)
        return is_noncollider_active(z)
    
    nodes = (i for i in range(len(path)))
    nodes = filter(lambda i: 0 < i < len(path) - 1, nodes)
    nodes = map(lambda i: get_triplet(i), nodes)
    nodes = map(lambda tup: is_node_active(*tup), nodes)
    return all(nodes)

def is_d_separated(g, source, target, Z=[]):
    m = get_descendants(g)
    paths = get_paths(g, source, target)
    for p in paths:
        if not is_path_active(g, p, Z, m):
            return True
    return False

In [21]:
descendant_map = get_descendants(g)
path = next(get_paths(g, 'D', 'S'))

r1 = is_path_active(g, path, [], descendant_map)
r2 = is_path_active(g, path, ['L'], descendant_map)
r3 = is_path_active(g, path, ['L', 'I'], descendant_map)

print(f'path = {path}')
print(f'Active(D, S | []) = {r1}')
print(f'Active(D, S | [L]) = {r2}')
print(f'Active(D, S | [L, I]) = {r3}')

path = ['D', 'G', 'I', 'S']
Active(D, S | []) = False
Active(D, S | [L]) = True
Active(D, S | [L, I]) = False


In [22]:
r1 = is_d_separated(g, 'D', 'S')
r2 = is_d_separated(g, 'D', 'S', ['L'])
r3 = is_d_separated(g, 'D', 'S', ['L', 'I'])

print(f'source=D, target=S')
print(f'I(D, S | []) = {r1}')
print(f'I(D, S | [L]) = {r2}')
print(f'I(D, S | [L, I]) = {r3}')

source=D, target=S
I(D, S | []) = True
I(D, S | [L]) = False
I(D, S | [L, I]) = True
